# DeepAR - Experiments

### DeepAR (Salinas et al., 2017)

The **DeepAR** algorithm uses autoregressive features that are used as the inputs to a recurrent neural network. The RNN uses LSTM cells as default but can also changed to GRU cells. In both cases previous time points are taken as input. 

- [`DeepAREestimator`](https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deepar.html)

In [1]:
# standard imports
import numpy as np
import pandas as pd

# json
import json

# gluon data
from gluonts.dataset.repository.datasets import get_dataset, dataset_recipes
from gluonts.dataset.util import to_pandas

# gluon imports
from gluonts.trainer import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator

# model imports
# from gluonts.model.deepstate import DeepStateEstimator
# from gluonts.model.deep_factor import DeepFactorEstimator
from gluonts.model.deepar import DeepAREstimator

import mxnet as mx
from pprint import pprint


### define main function ### 
def main(data="m4_weekly", seed=42, epochs=100, batches=50):
    
    dataset = get_dataset(data, regenerate=False)
    mx.random.seed(seed)
    np.random.seed(seed)
    
    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=epochs,      
        num_batches_per_epoch=batches,      
        learning_rate=1e-3,
    )
    
    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepAREstimator(
        trainer=trainer,
        cardinality=[cardinality],
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
        use_feat_static_cat=True   
    )
        
    predictor = estimator.train(dataset.train)
    
    forecast_it, ts_it = make_evaluation_predictions(
        dataset.test, predictor=predictor, num_eval_samples=100
    )
    
    agg_metrics, item_metrics = Evaluator()(
        ts_it, forecast_it, num_series=len(dataset.test)
    )
    
    metrics = ["MASE", "sMAPE", "MSIS", "wQuantileLoss[0.5]", "wQuantileLoss[0.9]"]
    output = {key: round(value, 8) for key, value in agg_metrics.items() if key in metrics}
    output["epochs"] = epochs
    return(output)


INFO:root:Using CPU


In [7]:
results = []

for i in range(100, 251, 50):
    print("Epochs:", i)
    res = main(data="m4_monthly", seed=42, epochs=i, batches=100)
    pprint(res)
    results.append(res)
    
results

Epochs: 100


INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_monthly.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 973443
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 100/100 [00:10<00:00,  9.73it/s, avg_epoch_loss=7.64]
INFO:root:Epoch[0] Elapsed time 10.317 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=7.635236
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 100/100 [00:08<00:00, 11.26it/s, avg_epoch_loss=7.65]
INFO:root:Epoch[1] Elapsed time 8.892 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=7.653496
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 100/100 [00:09<00:00, 10.60it/s, avg_epoch_loss=6.97]
INFO:root:Epoch[2] Elapsed time 9.445 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=6.970703
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 100/100 [00:10<00:00,  9.74it/s, avg_epoch_loss=6.78]
INFO:root:Epoch[3] Elapsed time 10.310

INFO:root:Epoch[36] Learning rate is 0.0005
100%|██████████| 100/100 [00:08<00:00, 11.96it/s, avg_epoch_loss=6.33]
INFO:root:Epoch[36] Elapsed time 8.372 seconds
INFO:root:Epoch[36] Evaluation metric 'epoch_loss'=6.329023
INFO:root:Epoch[37] Learning rate is 0.0005
100%|██████████| 100/100 [00:08<00:00, 11.39it/s, avg_epoch_loss=5.95]
INFO:root:Epoch[37] Elapsed time 8.786 seconds
INFO:root:Epoch[37] Evaluation metric 'epoch_loss'=5.948174
INFO:root:Epoch[38] Learning rate is 0.0005
100%|██████████| 100/100 [00:08<00:00, 11.75it/s, avg_epoch_loss=6.68]
INFO:root:Epoch[38] Elapsed time 8.516 seconds
INFO:root:Epoch[38] Evaluation metric 'epoch_loss'=6.678235
INFO:root:Epoch[39] Learning rate is 0.0005
100%|██████████| 100/100 [00:09<00:00, 11.09it/s, avg_epoch_loss=7.21]
INFO:root:Epoch[39] Elapsed time 9.031 seconds
INFO:root:Epoch[39] Evaluation metric 'epoch_loss'=7.214064
INFO:root:Epoch[40] Learning rate is 0.0005
100%|██████████| 100/100 [00:08<00:00, 11.63it/s, avg_epoch_loss=5.9

INFO:root:Epoch[72] Elapsed time 9.304 seconds
INFO:root:Epoch[72] Evaluation metric 'epoch_loss'=6.773929
INFO:root:Loading parameters from best epoch (52)
INFO:root:Epoch[73] Learning rate is 0.000125
100%|██████████| 100/100 [00:08<00:00, 11.32it/s, avg_epoch_loss=5.81]
INFO:root:Epoch[73] Elapsed time 8.843 seconds
INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=5.813602
INFO:root:Epoch[74] Learning rate is 0.000125
100%|██████████| 100/100 [00:08<00:00, 11.36it/s, avg_epoch_loss=6.98]
INFO:root:Epoch[74] Elapsed time 8.812 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=6.980258
INFO:root:Epoch[75] Learning rate is 0.000125
100%|██████████| 100/100 [00:08<00:00, 11.30it/s, avg_epoch_loss=6.77]
INFO:root:Epoch[75] Elapsed time 8.854 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=6.773364
INFO:root:Epoch[76] Learning rate is 0.000125
100%|██████████| 100/100 [00:08<00:00, 11.27it/s, avg_epoch_loss=5.99]
INFO:root:Epoch[76] Elapsed time 8.881 seconds
INFO:ro

{'MASE': 1.23645817,
 'MSIS': 22.85103734,
 'epochs': 100,
 'sMAPE': 0.13921232,
 'wQuantileLoss[0.5]': 0.12712547,
 'wQuantileLoss[0.9]': 0.09881872}
Epochs: 150


...

100%|██████████| 100/100 [00:08<00:00, 12.24it/s, avg_epoch_loss=5.32]
INFO:root:Epoch[181] Elapsed time 8.174 seconds
INFO:root:Epoch[181] Evaluation metric 'epoch_loss'=5.322238
INFO:root:Epoch[182] Learning rate is 5e-05
100%|██████████| 100/100 [00:08<00:00, 11.62it/s, avg_epoch_loss=6.77]
INFO:root:Epoch[182] Elapsed time 8.612 seconds
INFO:root:Epoch[182] Evaluation metric 'epoch_loss'=6.772326
INFO:root:Epoch[183] Learning rate is 5e-05
100%|██████████| 100/100 [00:09<00:00, 11.06it/s, avg_epoch_loss=6.55]
INFO:root:Epoch[183] Elapsed time 9.044 seconds
INFO:root:Epoch[183] Evaluation metric 'epoch_loss'=6.551254
INFO:root:Epoch[184] Learning rate is 5e-05
100%|██████████| 100/100 [00:08<00:00, 12.40it/s, avg_epoch_loss=5.76]
INFO:root:Epoch[184] Elapsed time 8.068 seconds
INFO:root:Epoch[184] Evaluation metric 'epoch_loss'=5.763025
INFO:root:Epoch[185] Learning rate is 5e-05
100%|██████████| 100/100 [00:08<00:00, 12.46it/s, avg_epoch_loss=6.23]
INFO:root:Epoch[185] Elapsed time

{'MASE': 1.02120132,
 'MSIS': 14.19714383,
 'epochs': 200,
 'sMAPE': 0.13583945,
 'wQuantileLoss[0.5]': 0.12079805,
 'wQuantileLoss[0.9]': 0.07770261}
Epochs: 250


INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_monthly.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 973443
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 100/100 [00:10<00:00,  9.12it/s, avg_epoch_loss=7.64]
INFO:root:Epoch[0] Elapsed time 10.997 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=7.635236
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 100/100 [00:11<00:00,  8.38it/s, avg_epoch_loss=7.65]
INFO:root:Epoch[1] Elapsed time 11.941 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=7.653496
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 100/100 [00:09<00:00, 10.03it/s, avg_epoch_loss=6.97]
INFO:root:Epoch[2] Elapsed time 9.978 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=6.970703
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 100/100 [00:09<00:00, 10.40it/s, avg_epoch_loss=6.78]
INFO:root:Epoch[3] Elapsed time 9.619

INFO:root:Epoch[36] Learning rate is 0.0005
100%|██████████| 100/100 [00:09<00:00, 10.90it/s, avg_epoch_loss=6.33]
INFO:root:Epoch[36] Elapsed time 9.179 seconds
INFO:root:Epoch[36] Evaluation metric 'epoch_loss'=6.329023
INFO:root:Epoch[37] Learning rate is 0.0005
100%|██████████| 100/100 [00:09<00:00, 10.58it/s, avg_epoch_loss=5.95]
INFO:root:Epoch[37] Elapsed time 9.461 seconds
INFO:root:Epoch[37] Evaluation metric 'epoch_loss'=5.948174
INFO:root:Epoch[38] Learning rate is 0.0005
  0%|          | 0/100 [00:02<?, ?it/s, avg_epoch_loss=5.65]


KeyboardInterrupt: 

In [ ]:
results = []

for i in range(100, 251, 50):
    print("Epochs:", i)
    res = main(data="m4_monthly", seed=43, epochs=i, batches=100)
    pprint(res)
    results.append(res)
    
results

In [ ]:
results = []

for i in range(100, 251, 50):
    print("Epochs:", i)
    res = main(data="m4_monthly", seed=43, epochs=i, batches=100)
    pprint(res)
    results.append(res)
    
results